In [2]:
import fmp
import pandas as pd

# Dow Jones: select stocks which have positive 1Y, 2Y and 3Y momentum

In [5]:
# Retrieve symbols and financials

sp500_tickers = fmp.sp500_symbols()
dow_tickers = fmp.dow_symbols()

# sp500_financials = dataSrc.financials(sp500_tickers)

df = fmp.extract_financials(dow_tickers)

# df = pd.read_csv('./R/data/dow_financials.csv')
# df = df.drop(['Unnamed: 0'], axis=1)
# df = df.dropna()

conditions = ((df["Revenue_Growth"] > 0)
              & (df["GPMargin"] > 0)
              & (df["EPS_Growth"] > 0)
              & (df["ROE"] > 0)
              & (df["DPS_Growth"] > 0)
              & (df["DivYield"] > 0))

df = df[conditions]

# Compute historical momentum

m12_momentums = []
m24_momentums = []
m36_momentums = []
count = 0
for symbol in df["symbol"]:
    count += 1
    m12_momentums.append(fmp.calculate_hist_momentum(symbol, 252))
    m24_momentums.append(fmp.calculate_hist_momentum(symbol, 504))
    m36_momentums.append(fmp.calculate_hist_momentum(symbol, 756))

df["m12_momentum"] = m12_momentums
df["m24_momentum"] = m24_momentums
df["m36_momentum"] = m36_momentums

positive_mom_stocks = df[(df['m12_momentum'] > 0) & (df['m24_momentum'] > 0)
                         & (df['m36_momentum'] > 0)]
positive_mom_stocks

CRM: 1/30
WBA: 2/30
V: 3/30
NKE: 4/30
UNH: 5/30
TRV: 6/30
VZ: 7/30
INTC: 8/30
WMT: 9/30
JNJ: 10/30
DIS: 11/30
MCD: 12/30
JPM: 13/30
CAT: 14/30
BA: 15/30
AMGN: 16/30
DOW: 17/30
AAPL: 18/30
GS: 19/30
CSCO: 20/30
MSFT: 21/30
HD: 22/30
PG: 23/30
MRK: 24/30
IBM: 25/30
HON: 26/30
KO: 27/30
CVX: 28/30
AXP: 29/30
MMM: 30/30


# Calculate stocks' discount rate from 52W high

In [9]:
watchlist = positive_mom_stocks[["symbol", "name"]].copy()

# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []
count = 0

for symbol in watchlist["symbol"]:
    count += 1
    print(f"{symbol}: {count}/{len(watchlist['symbol'])}")
    currentPrice = fmp.get_current_price(symbol)
    high = fmp.calculate_prev_max_high(symbol, 252)
    discount_pct = (currentPrice - high) / high * 100

    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)

watchlist["Current Price"] = currentPrices
watchlist["52W_High"] = highs
watchlist["Discount (%)"] = discounts

watchlist = watchlist.sort_values(by="Discount (%)")
watchlist

NKE: 1/7
TRV: 2/7
INTC: 3/7
JNJ: 4/7
CSCO: 5/7
MSFT: 6/7
HD: 7/7


,symbol,name,Current Price,52W_High,Discount (%)
7,INTC,Intel Corporation,53.73,68.489998,-21.550589
3,NKE,"NIKE, Inc.",163.67,174.380005,-6.141762
21,HD,"The Home Depot, Inc.",327.74,345.690002,-5.192514
9,JNJ,Johnson & Johnson,174.93,179.919998,-2.773454
5,TRV,"The Travelers Companies, Inc.",159.97,163.290000,-2.033192
20,MSFT,Microsoft Corporation,301.15,305.839996,-1.533480
19,CSCO,"Cisco Systems, Inc.",59.53,60.270000,-1.227808


# Consider buying if discounted more than 15%

In [10]:
buylist = watchlist[watchlist['Discount (%)'] < -15]
buylist

,symbol,name,Current Price,52W_High,Discount (%)
7,INTC,Intel Corporation,53.73,68.489998,-21.550589


# S&P500

In [12]:
# Retrieve symbols and financials

sp500_tickers = fmp.sp500_symbols()

df = fmp.extract_financials(sp500_tickers)

# df = pd.read_csv('./R/data/dow_financials.csv')
# df = df.drop(['Unnamed: 0'], axis=1)
# df = df.dropna()

conditions = ((df["Revenue_Growth"] > 0)
              & (df["GPMargin"] > 0)
              & (df["EPS_Growth"] > 0)
              & (df["ROE"] > 0)
              & (df["DPS_Growth"] > 0)
              & (df["DivYield"] > 0))

df = df[conditions]

# Compute historical momentum

m12_momentums = []
m24_momentums = []
m36_momentums = []
count = 0
for symbol in df["symbol"]:
    count += 1
    m12_momentums.append(fmp.calculate_hist_momentum(symbol, 252))
    m24_momentums.append(fmp.calculate_hist_momentum(symbol, 504))
    m36_momentums.append(fmp.calculate_hist_momentum(symbol, 756))

df["m12_momentum"] = m12_momentums
df["m24_momentum"] = m24_momentums
df["m36_momentum"] = m36_momentums

positive_mom_stocks = df[(df['m12_momentum'] > 0) & (df['m24_momentum'] > 0)
                         & (df['m36_momentum'] > 0)]
positive_mom_stocks

MMM: 1/505
ABT: 2/505
ABBV: 3/505
ABMD: 4/505
ACN: 5/505
ATVI: 6/505
ADBE: 7/505
AMD: 8/505
AAP: 9/505
AES: 10/505
AFL: 11/505
A: 12/505
APD: 13/505
AKAM: 14/505
ALK: 15/505
ALB: 16/505
ARE: 17/505
ALGN: 18/505
ALLE: 19/505
LNT: 20/505
ALL: 21/505
GOOGL: 22/505
GOOG: 23/505
MO: 24/505
AMZN: 25/505
AMCR: 26/505
AEE: 27/505
AAL: 28/505
AEP: 29/505
AXP: 30/505
AIG: 31/505
AMT: 32/505
AWK: 33/505
AMP: 34/505
ABC: 35/505
AME: 36/505
AMGN: 37/505
APH: 38/505
ADI: 39/505
ANSS: 40/505
ANTM: 41/505
AON: 42/505
AOS: 43/505
APA: 44/505
AAPL: 45/505
AMAT: 46/505
APTV: 47/505
ADM: 48/505
ANET: 49/505
AJG: 50/505
AIZ: 51/505
T: 52/505
ATO: 53/505
ADSK: 54/505
ADP: 55/505
AZO: 56/505
AVB: 57/505
AVY: 58/505
BKR: 59/505
BLL: 60/505
BAC: 61/505
BBWI: 62/505
BAX: 63/505
BDX: 64/505
BRK-B: 65/505
BBY: 66/505
BIO: 67/505
BIIB: 68/505
BLK: 69/505
BK: 70/505
BA: 71/505
BKNG: 72/505
BWA: 73/505
BXP: 74/505
BSX: 75/505
BMY: 76/505
AVGO: 77/505
BR: 78/505
BF-B: 79/505
CHRW: 80/505
COG: 81/505
CDNS: 82/505
CZR:

,symbol,name,exchange,sector,industry,marketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth,m12_momentum,m24_momentum,m36_momentum
4,ACN,Accenture plc,NYSE,Technology,Information Technology Services,216.240718,0.548620,0.315245,0.319888,0.087182,0.010323,3.520,1.449595,0.426898,0.709359,1.011799
11,A,"Agilent Technologies, Inc.",NYSE,Healthcare,Diagnostics & Research,53.890753,0.624099,0.203777,0.536470,0.223128,0.004280,0.762,0.704748,0.786632,1.421382,1.646351
15,ALB,Albemarle Corporation,NYSE,Basic Materials,Specialty Chemicals,28.539308,0.111984,0.133296,0.319454,3.270588,0.006352,1.550,0.277826,1.618910,2.875337,1.526975
16,ARE,"Alexandria Real Estate Equities, Inc.",NYSE,Real Estate,REIT—Office,31.769211,0.203399,0.064606,0.720226,64.250000,0.020910,4.360,0.261815,0.229567,0.370064,0.655104
18,ALLE,Allegion plc,NYSE,Industrials,Security & Protection Services,13.249026,0.373453,0.587647,0.433912,0.109244,0.009207,1.360,2.002203,0.470922,0.497769,0.694894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,UHS,"Universal Health Services, Inc.",NYSE,Healthcare,Medical Care Facilities,13.059048,0.516794,0.169798,0.887646,0.565041,0.003819,0.600,0.994461,0.401338,0.082334,0.225925
477,VMC,Vulcan Materials Company,NYSE,Basic Materials,Building Materials,24.280074,0.429297,0.110523,0.266741,0.214876,0.007923,1.450,0.852887,0.476640,0.280974,0.653415
488,WST,"West Pharmaceutical Services, Inc.",NYSE,Healthcare,Medical Instruments & Supplies,33.956409,0.847307,0.279328,0.393538,0.240196,0.001481,0.680,0.418274,0.717172,2.059592,2.950766
493,WHR,Whirlpool Corporation,NYSE,Consumer Cyclical,"Furnishings, Fixtures & Appliances",14.197954,0.240812,0.395190,0.218502,0.343523,0.023407,5.300,0.402728,0.319830,0.583870,0.826048


In [13]:
watchlist = positive_mom_stocks[["symbol", "name"]].copy()

# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []
count = 0

for symbol in watchlist["symbol"]:
    count += 1
    print(f"{symbol}: {count}/{len(watchlist['symbol'])}")
    currentPrice = fmp.get_current_price(symbol)
    high = fmp.calculate_prev_max_high(symbol, 252)
    discount_pct = (currentPrice - high) / high * 100

    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)

watchlist["Current Price"] = currentPrices
watchlist["52W_High"] = highs
watchlist["Discount (%)"] = discounts

watchlist = watchlist.sort_values(by="Discount (%)")
watchlist

ACN: 1/121
A: 2/121
ALB: 3/121
ARE: 4/121
ALLE: 5/121
AMT: 6/121
AWK: 7/121
AME: 8/121
APH: 9/121
ADI: 10/121
ANTM: 11/121
AOS: 12/121
AMAT: 13/121
ADM: 14/121
AIZ: 15/121
AVB: 16/121
BLL: 17/121
BAC: 18/121
BBY: 19/121
BLK: 20/121
BK: 21/121
AVGO: 22/121
BR: 23/121
COF: 24/121
CDW: 25/121
CE: 26/121
CSCO: 27/121
CL: 28/121
CMCSA: 29/121
COST: 30/121
CCI: 31/121
CSX: 32/121
CMI: 33/121
CVS: 34/121
DHI: 35/121
DHR: 36/121
DRI: 37/121
DFS: 38/121
DOV: 39/121
DRE: 40/121
ETN: 41/121
EFX: 42/121
EL: 43/121
RE: 44/121
EXC: 45/121
EXPD: 46/121
FAST: 47/121
FDX: 48/121
FBHS: 49/121
BEN: 50/121
FCX: 51/121
GRMN: 52/121
GD: 53/121
HIG: 54/121
HD: 55/121
ITW: 56/121
INTC: 57/121
ICE: 58/121
IP: 59/121
IPG: 60/121
IFF: 61/121
IVZ: 62/121
IRM: 63/121
JBHT: 64/121
SJM: 65/121
JNJ: 66/121
JCI: 67/121
JNPR: 68/121
KLAC: 69/121
KR: 70/121
LEG: 71/121
LLY: 72/121
LNC: 73/121
MLM: 74/121
MCHP: 75/121
MSFT: 76/121
MDLZ: 77/121
MPWR: 78/121
NDAQ: 79/121
NKE: 80/121
NSC: 81/121
NRG: 82/121
NUE: 83/121
NVDA

,symbol,name,Current Price,52W_High,Discount (%)
247,INTC,Intel Corporation,53.73,68.489998,-21.550589
204,FCX,Freeport-McMoRan Inc.,36.27,46.099998,-21.323207
439,TER,"Teradyne, Inc.",120.87,147.899994,-18.275859
282,LEG,"Leggett & Platt, Incorporated",48.62,59.160000,-17.816092
189,FDX,FedEx Corporation,267.09,319.899994,-16.508282
...,...,...,...,...,...
401,RMD,ResMed Inc.,294.40,294.500000,-0.033956
31,AMT,American Tower Corporation (REIT),300.89,300.980000,-0.029902
93,CDW,CDW Corporation,203.26,203.320000,-0.029510
400,RSG,"Republic Services, Inc.",126.10,126.110000,-0.007930


In [14]:
buylist = watchlist[watchlist['Discount (%)'] < -15]
buylist

,symbol,name,Current Price,52W_High,Discount (%)
247,INTC,Intel Corporation,53.73,68.489998,-21.550589
204,FCX,Freeport-McMoRan Inc.,36.27,46.099998,-21.323207
439,TER,"Teradyne, Inc.",120.87,147.899994,-18.275859
282,LEG,"Leggett & Platt, Incorporated",48.62,59.160000,-17.816092
189,FDX,FedEx Corporation,267.09,319.899994,-16.508282
399,RF,Regions Financial Corporation,20.04,23.809999,-15.833680
412,STX,Seagate Technology Holdings plc,89.50,106.220000,-15.740915
387,PHM,"PulteGroup, Inc.",54.20,63.910000,-15.193240


In [ ]:
#!jupyter nbconvert --to script --no-prompt analysis.ipynb